# Optimización multi-objetivo
Hasta ahora nos hemos enfrentado a problemas de optimización en los que el objetivo era optimizar cierta función, pero dicho objetivo era único:

- Maximizar el valor de los objetos que se incluyen en la mochila
- Minimizar el número de reinas que se amenazan en el _problema de las n-reinas_
- Minimizar la distancia recorrida en el _problema del viajante de comercio_
- Maximizar el número de 1's en un array binario en el _problema MaxOnes_

Para resolverlos usando un algoritmo evolutivo, hay que definir la codificación de las soluciones, elegir los operadores que se van a utilizar para generar nuevas, y establecer cuál va a ser la función de fitness que guíe al algoritmo en la búsqueda de mejores soluciones. Sin embargo, hay problemas que requieren de un enfoque multi-objetivo porque el problema al que nos enfrentamos consta de varios objetivos de optimización. En este caso, la principal diferencia es que la función de _fitness_ que decidamos utilizar ya no nos devuelve un único valor real, sino que nos devolverá varios, dependiendo del número de objetivos que formen el problema de optimización a resolver:

$$
f:\mathbb{S} \rightarrow \mathbb{R}^n
$$

Este pequeño cambio solo afecta a una parte concreta de los algoritmos evolutivos: **los mecanismos de selección**. Como ya hemos visto en temas anteriores, los algoritmos evolutivos basan su funcionamiento, entre otras cosas, en el mecanismo de selección que usan tanto para elegir a las soluciones que sirven de semilla para generar otras nuevas como al mecanismo para determinar qué soluciones de la población sobreviven y pasan a la siguiente generación del proceso evolutivo.

Por tanto, la diferencia entre enfrentarse a un problema multi-objetivo con respecto a uno con objetivo único radica, precisamente, en los mecanismos de selección que se utilicen, básicamente porque son éstos los que tienen en cuenta el _fitness_ de los individuos y, realmente, es el único cambio entre ambos enfoques.

A lo largo de la historia de los algoritmos evolutivos se han ido diseñando diferentes enfoques multi-objetivo. Más adelante se describen dos de los más usados: NSGA-II y SPEA2. Pero antes, veamos qué es la **dominancia** en espacios multi-dimensionales y qué se conoce como el **frente de Pareto**.


## Dominancia en espacios multi-dimensionales y frentes de Pareto
Formalmente, un **problema de optimización multi-objetivo** se define como maximizar/minimizar:

$$
f(x)=(f_1(x),f_2(x),\ldots,f_k(x))
$$

Para establecer un orden de calidad en las soluciones de un espacio multi-dimensiona, se define lo que se conoce como dominancia. Se dice que un vector $\overrightarrow{u}$ domina a otro $\overrightarrow{v}$, denotándose $\overrightarrow{u}\preceq \overrightarrow{v}$,  sí y solo sí (suponiendo maximización):

$$
{\Large \forall}i\in\left\{1, 2, \ldots, k \right\}\mid f_i(\overrightarrow{u}) \geq f_i(\overrightarrow{v}) \wedge \exists j \in \left\{1, 2, \ldots, k \right\} \mid f_j(\overrightarrow{u}) > f_j(\overrightarrow{v})
$$

Es decir, **una solución domina a otra si es mejor o igual en todos los objetivos y al menos mejor en uno de ellos**. Por ejemplo, en la siguiente figura, si suponemos que hay que maximizar ambos objetivos, tanto $u$ como $v$ dominan a $w$, sin embargo las dos primeras no son dominadas por ninguna otra solución.

![Dominancia](https://www.researchgate.net/profile/Jamal_Toutouh/publication/323005390/figure/fig7/AS:591681388830721@1518079244305/Dominance-in-multi-objective-optimization.png)

Una solución que no está dominada por ninguna otra se conoce como **solución Pareto-óptima**. El conjunto de todas las soluciones no dominadas $X^*\subset X$ es el **conjunto Pareto-óptimo** y forman la solución óptima del problema multiobjetivo. Por su parte, los valores de _fitness_ de las soluciones del dicho conjunto forman lo que se conoce como **Frente de Pareto**.

Debido a que no suele existir una única solución optima, sino que existe un conjunto (a veces infinito) de soluciones No-Dominadas que forman la Frontera de Pareto, el objetivo de la optimización multi-objetivo es encontrar una aproximación del frente de Pareto de la mayor calidad posible.

A continuación se presenta uno de los algoritmos evolutivos multi-objetivo más conocidos y utilizados: el **NSGA-II**.

## Non-dominated Sorting Genetic Algorithm II (NSGA-II)
El algoritmo **NSGA-II** pertenece a la familia de los algoritmos evolutivos y sus características principales se resumen en:

1. Usa un enfoque elitista. Es decir, las mejores soluciones pasan automáticamente a la siguiente generación
2. Utiliza un mecanismo explícito para conservar la diversidad de la población. Se le conoce como **crowding distance**.
3. Basa su selección en las soluciones no dominadas

Su modo de funcionamiento es el siguiente:

1. Se hace una ordenación de las soluciones padre e hijas en base a su no-dominancia. De esta forma se agrupan en varios conjuntos: soluciones no-dominadas, soluciones dominadas por una solución, por dos soluciones, etc.
2. Se completa la nueva población añadiendo las soluciones a partir de los conjuntos calculados anteriormente.
3. Si un conjunto no se puede añadir completamente, se reordenan las soluciones que forman parte de dicho conjunto mediante la métrica **crowding-distance**, y se seleccionan tantos como individuos falten para completar la próxima población.
4. Crea una población de descendientes a partir de esta nueva población utilizando una selección por torneo (se comparan primero por el número de soluciones que las dominan y, si es igual, se compara por sus distancias de _crowding_), operadores de cruce y de mutación.

![NSGA-II](https://pythonhealthcaremodelling.files.wordpress.com/2019/01/nsga.png)

Para ver cómo funciona el algoritmo y resolver algunos problemas vamos a utilizar una librería de optimización multiobjetivo para Python, denominada [Platypus](https://platypus.readthedocs.io/en/latest/#). Esta librería trae algunas funciones de elevada complejidad de optimización que suelen usarse como benchmarks para los algoritmos de optimización multiobjetivo. En el siguiente ejemplo se optimizará la función conocida como [DTLZ2](https://sop.tik.ee.ethz.ch/download/supplementary/testproblems/dtlz2/index.php):

In [0]:
# Instalación de la librería en el entorno de ejecución de la notebook
!pip install platypus-opt

In [0]:
from platypus import NSGAII, DTLZ2

# definición del problema
problem = DTLZ2()

# se instancia el algoritmo a resolver
algorithm = NSGAII(problem)

# se lanza el algoritmo especificando el número máximo de evaluaciones
algorithm.run(10000)

# plot the results using matplotlib
import matplotlib.pyplot as plt

plt.scatter([s.objectives[0] for s in algorithm.result],
            [s.objectives[1] for s in algorithm.result])
plt.xlim([0, 1.1])
plt.ylim([0, 1.1])
plt.xlabel("$f_1(x)$")
plt.ylabel("$f_2(x)$")
plt.show()

Vamos a usar la misma librería para resolver el problema de la mochila ilimitada y multi-objetivo:

In [0]:
import random

from platypus import Problem, Integer, NSGAII
import matplotlib.pyplot as plt


class MochilaMulti(Problem):
    def __init__(self, pesos, valores, volumenes, W):
        super(MochilaMulti, self).__init__(nvars=len(valores), nobjs=2, nconstrs=1)
        self.types[:] = [Integer(0, W // min(pesos))] * len(valores)
        self.constraints[:] = ">=0"
        self.directions[:] = [Problem.MAXIMIZE, Problem.MINIMIZE]
        self.valores = valores
        self.volumenes = volumenes
        self.W = W
        self.pesos = pesos

    def evaluate(self, solution):
        seleccion = solution.variables
        solution.objectives[:] = [sum(list(map(lambda x: x[0]*x[1], zip(seleccion, self.valores)))),
                                  sum(list(map(lambda x: x[0]*x[1], zip(seleccion, self.volumenes))))]
        solution.constraints[:] = [sum(list(map(lambda x: x[0]*x[1], zip(seleccion, self.pesos)))) - self.W]


# Instancia aleatoria del problema
N = 60
wi = tuple([random.randint(1, 100) for _ in range(N)])
vi = tuple([random.randint(1, 100) for _ in range(N)])
voli = tuple([random.randint(1, 100) for _ in range(N)])
W = sum(wi) // 2

problema = MochilaMulti(wi, vi, voli, W)
algorithm = NSGAII(problema, population_size=100)
algorithm.run(10000)

plt.scatter([s.objectives[0] for s in algorithm.result],
            [s.objectives[1] for s in algorithm.result])
plt.xlabel("valor")
plt.ylabel("volumen")
plt.show()


Para terminar con este tema, a continuación se realiza una comparativa de los conjuntos de soluciones no dominadas obtenidas tanto por el algoritmo que acabamos de ver, **NSGA-II**, como con otro algoritmo de optimización multi-objetivo **SPEA2**:

In [0]:
from platypus import *

algoritmos = [NSGAII, SPEA2]
problemas = [MochilaMulti(wi, vi, voli, W)]

with ProcessPoolEvaluator() as evaluator:
        results = experiment(algoritmos, problemas, seeds=1, nfe=10000, evaluator=evaluator)

fig = plt.figure(figsize=(45, 16))

for i, algorithm in enumerate(six.iterkeys(results)):
        result = results[algorithm]["MochilaMulti"][0]
        
        ax = fig.add_subplot(2, 5, i+1)
        ax.scatter([s.objectives[0] for s in result],
                   [s.objectives[1] for s in result])
        ax.set_title(algorithm)
        plt.xlabel('valor')
        plt.ylabel('volumen')
    
plt.show()

# Hibridación con otros algoritmos: Algoritmos Meméticos
Muchos problemas complejos se pueden descomponer en varias partes, entre las cuales habrá algunas partes para las que se conozcan algoritmos de optimización exactos o buenas heurísticas para resolverlas. En estos casos parece que tiene sentido usar una combinación de los métodos más adecuados para los distintos sub-problemas.

En general, no existen métodos resolutores de problemas generales exitosos y eficientes. El creciente número de pruebas empíricas y algunos resultados teóricos, como el teorema de [No Free Lunch (NFL)](https://en.wikipedia.org/wiki/No_free_lunch_in_search_and_optimization), respaldan firmemente este punto de vista. Desde el punto de vista de la computación evolutiva, esto implica que los EAs no exhiben el desempeño sugerido en la década de 1980. En otras palabras, un algoritmo evolutivo particular puede ser muy bueno resolviendo un determinado problema, pero no será capaz de superar a una búsqueda aleatoria en la totalidad de tipos de problemas.

![No Free Lunch](https://cdn-images-1.medium.com/max/800/1*0QP3OeK7BAOWGlUcDG6VSw.png)

En la práctica, aplicamos con frecuencia un algoritmo evolutivo a un problema en el que hay una cantidad considerable de experiencia de usuario y conocimientos adquiridos con esfuerzo. En tales casos, la utilización de esta información en forma de operadores especializados o de buenas soluciones puede resultar beneficiosa para el rendimiento, siempre que se tenga cuidado de no sesgar demasiado la búsqueda en detrimento de la generación de nuevas soluciones. En estos casos, se suele experimentar que la combinación de un método evolutivo y un método heurístico -un algoritmo evolutivo híbrido- funciona mejor que cualquiera de sus algoritmos "parentales" por sí solos.

Hay varias formas de incorporar conocimiento específico del problema a un algoritmo evolutivo. A continuación veremos algunas de ellas.

## Inicialización 'inteligente' o heurística
Una de las formas más obvias de incoporar conocimiento específico del problema es en la fase de inicialización del algoritmo evolutivo. Como hemos visto, la creación de la población inicial de soluciones en un AE es un proceso aleatorio. Sin embargo, introduciendo algunas soluciones conocidas del problema dentro de la población inicial del algoritmo podemos obtener algunas ventajas:

1. Es posible evitar 'reinventar la rueda' utilizando soluciones conocidas. De esta forma evitamos que el algoritmo tenga que evolucionar las soluciones iniciales hacia ellas.
2.  Una población inicial no aleatoria puede dirigir la búsqueda hacia zonas del espacio de soluciones prometedoras.
3. Como siempre, hay que encontrar el equilibrio entre una población totalmente aleatoria y una formada por soluciones conocidas. Como ya hemos comentado, es muy importante mantener la diversidad en los individuos explorados para evitar que el proceso de búsqueda se estanque en un óptimo global

Para cambiar la función de inicialización e incluir conocimiento específico del problema en la población, podemos utlizar técnicas como las siguientes:

- Sembrar la población con una o más buenas soluciones previamente conocidas, que surjan de otras técnicas. Estas técnicas abarcan desde el ensayo y error humano hasta el uso de heurísticas  _greedy_ altamente especializadas que utilizan información específica de cada caso.
- En la inicialización selectiva se crea un gran número de soluciones aleatorias y luego se selecciona la población inicial de entre ellas. Otras alternativas incluyen la selección de un conjunto basado no sólo en lel _fitness_ sino también en la diversidad para maximizar la cobertura del espacio de búsqueda.
- Realizar una búsqueda local a partir de cada miembro de la población inicial, de modo que la población inicial consista en un conjunto de puntos que sean localmente óptimos con respecto a algún operador de movimientos.
- Usando uno o más de los métodos anteriores para identificar una (o más) buenas soluciones, y luego clonándolas y aplicando una mutación de alta tasa (mutación masiva) para producir un número de individuos en las proximidades del punto de partida.

## Hibridación en la variación: cruce y mutación 'inteligentes'
Como su nombre indica, consiste en introducir sesgos en el proceso de mutación y cruce, introduciendo de esta forma información específica del problema.

Un ejemplo de este tipo de hibridación sería el siguiente: supongamos que estamos usando un algoritmo genético para evolucionar distintas selecciones de las características (_features_) que se emplearán para entrenar un clasificador. Podríamos 'influenciar' la mutación para obtener selecciones de _features_ más compactas, aumentando la probabilidad de que se pase de un valor de "usar" a otro de "no usar" con respecto a la operación contraria.

Se han desarrollado multitud de operadores de mutación y cruce específicos para algunos de los problemas clásicos de optimización que hemos visto durante este curso.

## Búsqueda local después de la variación
El uso más común de la hibridación dentro de los AEs, y el que mejor se ajusta al concepto de Dawkins del meme, es a través de la aplicación de una o más fases de mejora local a cada uno de los miembros individuales de la población durante el ciclo evolutivo, es decir, la búsqueda local actúa sobre soluciones completas creadas por mutación o recombinación. Esto puede ocurrir en diferentes lugares del ciclo, es decir, antes o después de la selección o después del cruce y/o la mutación. 

Por ejemplo, podríamos aplicar un descenso de gradiente a cada uno de los miembros de la población resultante de aplicar cruce y mutación. De esta forma no solo estamos añadiendo diversidad en la población, sino que estamos aplicando una pequeña mejora a cada una de las soluciones candidatas.


## Pseudo-código de un algoritmo memético


```
1. Inicializar población <- Hibridación
2. Evaluar los candidatos de la población
3. Mientras no se cumpla la condición de parada
    3.1. Seleccionar padres
    3.2. Recombinación (cruce) <- Hibridación
    3.3. Mutación <- Hibridación
    3.4. Búsqueda local de cada candidato
    3.5. Evaluación de nuevos candidatos
```

